In [ ]:
# – Warehouse Queries & Insights
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns

load_dotenv()

engine = create_engine(
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}"
    f"@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

# ─── 1. Top channels by message volume ──────────────────────────────────────────
top_channels = pd.read_sql("""
    SELECT channel_name, total_posts, avg_views
    FROM public_marts.dim_channels
    ORDER BY total_posts DESC
    LIMIT 10
""", engine)

print("Top 10 channels by post count:")
display(top_channels)

plt.figure(figsize=(10,5))
sns.barplot(data=top_channels, x="total_posts", y="channel_name")
plt.title("Top 10 Telegram Channels by Message Count")
plt.show()

# ─── 2. Image category vs average views (Task 3 insight) ────────────────────────
views_by_category = pd.read_sql("""
    SELECT 
        d.image_category,
        COUNT(*) as count,
        AVG(m.view_count) as avg_views,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY m.view_count) as median_views
    FROM public_marts.fct_image_detections d
    JOIN public_marts.fct_messages m USING (message_id, channel_key)
    GROUP BY d.image_category
    ORDER BY avg_views DESC
""", engine)

print("\nImage category performance:")
display(views_by_category)

plt.figure(figsize=(8,5))
sns.barplot(data=views_by_category, x="image_category", y="avg_views")
plt.title("Average Views by Image Category")
plt.xticks(rotation=45)
plt.show()

# ─── 3. Posting activity over time ──────────────────────────────────────────────
activity = pd.read_sql("""
    SELECT 
        d.year,
        d.month,
        d.month_name,
        COUNT(*) as message_count
    FROM public_marts.fct_messages f
    JOIN public_marts.dim_dates d USING (date_key)
    GROUP BY d.year, d.month, d.month_name
    ORDER BY d.year, d.month
""", engine)

plt.figure(figsize=(12,5))
sns.lineplot(data=activity, x="month_name", y="message_count", hue="year", marker="o")
plt.title("Monthly Posting Volume Trend")
plt.show()

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "marts.dim_channels" does not exist
LINE 3:     FROM marts.dim_channels
                 ^

[SQL: 
    SELECT channel_name, total_posts, avg_views
    FROM marts.dim_channels
    ORDER BY total_posts DESC
    LIMIT 10
]
(Background on this error at: https://sqlalche.me/e/20/f405)